In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torch.nn.functional as F

from torch.autograd import Variable
from torchvision import datasets, transforms

import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns

from tkmodel.TwoCUM_copy import TwoCUMfittingConc
from tkmodel.TwoCUM_copy import TwoCUM

from ml_model.data_generation import uterus
from ml_model.arg_extractor import get_args
from ml_model.experiment_build import ExperimentBuilder
from ml_model.model_architectures import BasicNet
import os 

In [ ]:
batch_train = args.batch_size

batch_val = args.batch_size_test

num = 100000

uterus_data = uterus(num)
train, val, test, scaler = uterus_data.create_data(batch_train, batch_val, shuffle = True)

custom_conv_net = BasicNet()

mri_experiment = ExperimentBuilder(network_model=custom_conv_net,
                                    experiment_name=args.experiment_name,
                                    num_epochs=args.num_epochs,
                                    weight_decay_coefficient=args.weight_decay_coefficient,
                                    pk_weight = args.pk_weight, 
                                    curve_weight = args.curve_weight,
                                    lr = args.lr,
                                    train_data=train, 
                                    val_data=val,
                                    test_data=test,
                                    scaler = scaler)  # build an experiment object